In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
cd /Users/brendan/Documents/projects/mpa_project/gfw_research

/Users/brendan/Documents/projects/mpa_project/gfw_research


In [4]:
import numpy as np
import pandas
from datetime import datetime
import matplotlib.pyplot as plt
import os
import pywdpa
import geopandas
import contextily as ctx
from shapely import geometry
from shapely import ops
import pretty_html_table
import cProfile

import util

pandas.set_option('display.max_columns', None)
pandas.set_option('display.max_rows', None)

/opt/homebrew/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [5]:
DATA_PATH = "/Users/brendan/Documents/projects/mpa_project/gfw_research/data/"
FILENAME = "mpatlas_20201223_clean/mpatlas_20201223_clean.shp"

#mpas = geopandas.read_file(DATA_PATH + FILENAME)
# util.load_mpatlas_mpas()
mpas = geopandas.read_parquet("data/mpas/mpatlas.parquet")
old_crs = mpas.crs
selected_areas = pandas.read_csv("data/mpas/selected_areas.csv")


In [6]:
def load_year(year):
    year = str(year)
    print(f'\r reading year {year}', end='')
    return pandas.read_parquet("data/points/" + year + ".parquet")

In [7]:
points_by_year = {
    year : load_year(year)
    for year in range(2012, 2021)
}

 reading year 2020

In [8]:
# sample the points here before converting to geo
# use the sampled points to find the mmsi that are interesting
# only then should we get all of the points of relevent mmsi

SAMPLE_RATIO = 100
points_sampled = []
for year, points in points_by_year.items():
    print(f'\rsampling {str(year)}... ', end='')
    points_sampled.append(points.sample(frac=1/SAMPLE_RATIO))
print('done.')
points_sampled = pandas.concat(points_sampled)
print("converting to geo...  ", end='')
geopoints_sampled = util.convert_to_geo(points_sampled, box=True)
print("done.")


sampling 2020... done.
converting to geo...  done.


In [9]:
merged = mpas.merge(selected_areas[["mpa_id", "wdpa_id"]], on=["mpa_id", "wdpa_id"], how="right")
merged['geography_area'] = (merged.to_crs(epsg=6933).area / 1000000).astype('int32')
merged.to_crs(old_crs)
merged.at[16, "implementa"] = '2015-01-01'

In [10]:
merged['mmsi_count'] = 0
merged['in_pre'] = 0
merged['out_pre'] = 0
merged['in_post'] = 0
merged['out_post'] = 0

In [24]:
import util
folder_prefix = f'plots/{np.datetime64("now")}'

plot=True

tables = pandas.DataFrame()
summed_tables = pandas.DataFrame()
for i, row in merged.iterrows():
    
    name = row["name"].replace('/', '').replace('\\', '')
    
    print(f'\r{i}/{merged.shape[0]} {name}                            ', end='')

    date = row["implementa"]
    if date is None:
        date = str(row["status_yea"])
    
    folder = folder_prefix + f'/mpas/{name}/'
    
    returned = util.analyze_mpa(
        geopoints_sampled, points_by_year, merged.iloc[i:i+1], date, name, verbose=False,
        plot=plot, folder=folder)
    
    
    if returned is not None:
        table, points_of_mpa_ships = returned

        
        for key in ['in_pre', 'out_pre', 'in_post', 'out_post']:
            merged.at[i, key] = table[key].sum()
        merged.at[i, 'mmsi_count'] = table['mmsi'].count()
        

# Populate the summed tables percentages columns
# summed_tables['in_pre_p'] = summed_tables['in_pre'] / (summed_tables['in_pre'] + summed_tables['out_pre'])
# summed_tables['in_post_p'] = summed_tables['in_post'] / (summed_tables['in_post'] + summed_tables['out_post'])
# summed_tables['decrease'] = (summed_tables['in_pre_p'] - summed_tables['in_post_p'])/(summed_tables['in_pre_p'])
# summed_tables = summed_tables.fillna(0)


# if plot:
#     # Save to html files
#     html_table = pretty_html_table.build_table(summed_tables.sort_values('decrease', ascending=False), 'blue_light')
#     with open(folder_prefix+'/summed_effort_by_decrease.html', 'w') as f:
#         f.write(html_table)


#     html_table = pretty_html_table.build_table(summed_tables.sort_values('mmsi', ascending=False), 'blue_light')
#     with open(folder_prefix+'/summed_effort_by_mmsi.html', 'w') as f:
#         f.write(html_table)

136/151 KKPN KEPULAUAN ANAMBAS DAN LAUT SEKITARNYA                 eauty                                                            
150/151 Cagayancillo                                                                                

In [14]:
merged.sort_values('geography_area', ascending=False)

,mpa_id,wdpa_id,wdpa_pid,name,country,sovereign,designatio,designat_1,designat_2,iucn_categ,status,status_yea,is_mpa,implemente,implementa,no_take,no_take_ar,rep_m_area,calc_m_are,rep_area,calc_area,gov_type,mgmt_auth,mgmt_plan_,fishing,access,constancy,permanence,SHAPE_Leng,SHAPE_Area,geometry,geography_area,mmsi_count,in_pre,out_pre,in_post,out_post
0,68808362,555624907,None,Marae Moana,COK,COK,Marine Managed Area,Marine Managed Area,National,VI,Designated,2017,0,1,None,Not Reported,0.000000e+00,1673000.00,1.706212e+06,1673000.00,0.000000e+00,Collaborative governance,Marae Moana Council,Management plan currently under development,Yes,Yes,Unknown,Unknown,116.579516,143.887769,"MULTIPOLYGON (((-157.87416 -21.93861, -157.874...",1706196,108,94362,404338,312273,855987
1,9047,555624810,555624810_A,Ross Sea Region [General Protection Zone (i) R...,ABNJ,ABNJ,Marine Protected Area (CCAMLR),Marine Protected Area (CCAMLR),International,Not Reported,Designated,2017,1,1,2017-12-01,All,1.555851e+06,1555851.00,9.737040e+05,1555851.00,1.521512e+06,Joint governance,Commission for the Convention on Conservation ...,https://www.ccamlr.org/en/measure-91-05-2016,No,Unknown,Unknown,Permanent,443.363546,554.382341,"MULTIPOLYGON (((180.00000 -84.34113, 179.95204...",1555942,11,5876,182278,763,92471
2,15625,555577562,555577562,Natural Park of the Coral Sea,NCL,FRA,Parc Naturel,Natural Park,National,VI,Designated,2014,1,1,2018-03-01,None,2.800000e+04,1292962.00,1.294950e+06,1292967.00,1.291642e+06,Sub-national ministry or agency,New Caledonian Government,http://www.affmar.gouv.nc/portal/page/portal/a...,Unknown,Unknown,Unknown,Unknown,71.550285,112.435505,"POLYGON ((164.16940 -15.91227, 164.36711 -16.1...",1289995,29,198945,35399,119976,21428
3,9259,555547601,None,South Georgia and South Sandwich Islands,SGS,GBR,Marine Protected Area,Marine Protected Area,National,VI,Designated,2012,1,1,None,None,0.000000e+00,1240000.00,1.242753e+06,1240000.00,1.246715e+06,Federal or national ministry or agency in charge,Government of South Georgia and South Sandwich...,http://www.gov.gs/docsarchive/Environment/Mari...,Some Restrictions,Yes,Year-round,Unknown,60.235013,182.193915,"POLYGON ((-30.51266 -53.40252, -30.51183 -53.4...",1246714,25,0,0,145379,297806
4,68819490,555547601,555547601,South Georgia and South Sandwich Islands Marin...,SGS,GBR,Marine Protected Area,Marine Protected Area,National,VI,Designated,2012,1,1,None,Part,3.028460e+05,1240000.00,1.065899e+06,1240000.00,1.069871e+06,Federal or national ministry or agency,Government of South Georgia and South Sandwich...,http://www.sgisland.gs/download/MPA/MPA%20Plan...,Unknown,Unknown,Unknown,Unknown,87.941454,180.808019,"MULTIPOLYGON (((-36.19410 -54.46968, -36.19298...",1237120,25,0,0,143593,299591
5,68808390,0,None,Papahānaumokuākea 2016 Expansion,USA,USA,Marine National Monument,Marine National Monument,National,None,Designated,2016,1,1,None,All,0.000000e+00,0.00,0.000000e+00,1508870.00,0.000000e+00,Partnership,"Department of Commerce, the Department of the ...",None,Some Restrictions,Unknown,Year-round,Permanent,99.340151,103.652013,"MULTIPOLYGON (((180.00000 31.46826, 179.99992 ...",1146553,6,954,11818,934,71205
6,68818911,555692282,555692282,Périmètre De Protection De La Réserve Naturell...,ATF,FRA,Périmètre De Protection D’Une Réserve Naturell...,Protected Perimeter Around A National Nature R...,National,Not Reported,Designated,2017,1,1,None,Not Reported,0.000000e+00,0.00,9.896110e+05,1010780.00,9.896110e+05,Federal or national ministry or agency,Not Reported,Not Reported,Unknown,Unknown,Unknown,Unknown,120.907548,109.836846,"MULTIPOLYGON (((80.94849 -40.79863, 80.93521 -...",988912,3,244,28675,801,14893
7,68808360,345888,345888_F,Terres Australes Françaises [Périmètre de prot...,ATF,FRA,Réserve Naturelle Nationale,Protection zone around Nature Reserve,National,VI,Designated,2017,0,1,2016-12-16,None,0.000000e+00,989787.00,9.892400e+05,989787.00,9.892400e+05,Federal or national ministry or agency,Terr

In [25]:
merged.drop(columns='geometry').sort_values('geography_area', ascending=False).to_csv(path_or_buf='selected_areas_with_effort.csv')